# Detecting Sentiment Based on OpenAi Solution

https://openai.com/blog/unsupervised-sentiment-neuron/

This solution consists on using a multiplicative LSTM (mLSTM) to simply predict the next charactere on product reviews. After training the recurrent network, use it's hidden units to train a classifier.

In my case, I lack the resources and time available in OpenAi (as long as the knowledge), so I am going to do my best with a few days of training on a GeForce GTX 1060 instead of "one month across four NVIDIA Pascal GPUs [...] processing 12,500 characters per second."

Instead of using amazon reviews, I will use a portuguese language review dataset obtained from https://www.kaggle.com/olistbr/brazilian-ecommerce.

This project was made for the natural language processing discipline, UFPELs computer science course.

In [1]:
import os
import sys
import pandas as pd
raw_data = "../data/raw/"

reviews = pd.read_csv(os.path.join(raw_data, "olist_order_reviews_dataset.csv"), 
                      usecols=["review_id", "review_score", "review_comment_title", "review_comment_message"]).fillna("")
reviews = reviews[reviews.review_comment_message != ""]
reviews.head()

,review_id,review_score,review_comment_title,review_comment_message
3,e64fb393e7b32834bb789ff8bb30750e,5,,Recebi bem antes do prazo estipulado.
4,f7c4243c7fe1938f181bec41a392bdeb,5,,Parabéns lojas lannister adorei comprar pela I...
9,8670d52e15e00043ae7de4c01cc2fe06,4,recomendo,aparelho eficiente. no site a marca do aparelh...
12,4b49719c8a200003f700d3d986ea1a19,4,,"Mas um pouco ,travando...pelo valor ta Boa.\r\n"
15,3948b09f7c818e2d86c9a546758b2335,5,Super recomendo,"Vendedor confiável, produto ok e entrega antes..."


In [2]:
reviews.groupby("review_score").review_id.count()

review_score
1     9179
2     2229
3     3665
4     6034
5    20646
Name: review_id, dtype: int64

# Define Multiplicative LSTM

Model of recurrent neural network used by OpenAi people (https://arxiv.org/abs/1609.07959). Model obtained from https://discuss.pytorch.org/t/implementation-of-multiplicative-lstm/2328/5.

In [3]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

class mLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, embed_size, output_size):
        super(mLSTM, self).__init__()

        self.hidden_size = hidden_size
        # input embedding
        self.encoder = nn.Embedding(input_size, embed_size)
        # lstm weights
        self.weight_fm = nn.Linear(hidden_size, hidden_size)
        self.weight_im = nn.Linear(hidden_size, hidden_size)
        self.weight_cm = nn.Linear(hidden_size, hidden_size)
        self.weight_om = nn.Linear(hidden_size, hidden_size)
        self.weight_fx = nn.Linear(embed_size, hidden_size)
        self.weight_ix = nn.Linear(embed_size, hidden_size)
        self.weight_cx = nn.Linear(embed_size, hidden_size)
        self.weight_ox = nn.Linear(embed_size, hidden_size)
        # multiplicative weights
        self.weight_mh = nn.Linear(hidden_size, hidden_size)
        self.weight_mx = nn.Linear(embed_size, hidden_size)
        # decoder
        self.decoder = nn.Linear(hidden_size, output_size)

    def forward(self, inp, h_0, c_0):
        # encode the input characters
        inp = self.encoder(inp)
        # calculate the multiplicative matrix
        m_t = self.weight_mx(inp) * self.weight_mh(h_0)
        # forget, input and output gates
        f_g = torch.sigmoid(self.weight_fx(inp) + self.weight_fm(m_t))
        i_g = torch.sigmoid(self.weight_ix(inp) + self.weight_im(m_t))
        o_g = torch.sigmoid(self.weight_ox(inp) + self.weight_om(m_t))
        # intermediate cell state
        c_tilda = torch.tanh(self.weight_cx(inp) + self.weight_cm(m_t))
        # current cell state
        cx = f_g * c_0 + i_g * c_tilda
        # hidden state
        hx = o_g * torch.tanh(cx)

        out = self.decoder(hx.view(1,-1))

        return out, hx, cx

    def init_hidden(self):
        h_0 = Variable(torch.zeros(1, self.hidden_size)).cuda()
        c_0 = Variable(torch.zeros(1, self.hidden_size)).cuda()
        return h_0, c_0

In [4]:
import random

def generate_chunk(predict_len=200, temperature=0.5):
    prime_str = "O produto"  
    hidden, cell = rnn.init_hidden()
    prime_input = char_tensor(prime_str).cuda()
    predicted = prime_str

    # prime_str é o texto inicial que o gerado irá completar
    for p in range(len(prime_str) - 1):
        _, hidden, cell = rnn(prime_input[p], hidden, cell)
    inp = prime_input[-1]
    
    for p in range(predict_len):
        output, hidden, cell = rnn(inp, hidden, cell)
        
        # Usa a temperatura para amostrar a distribuição e escolher a saída probabilísticamente
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        
        # Adiciona o caracter predito à string de saída
        predicted_char = chr(top_i)
        predicted += predicted_char
        inp = char_tensor(predicted_char).cuda()

    return predicted

In [5]:
# Converte string para uma lista de inteiros
from unidecode import unidecode

def tensor2char(tensor, temperature=0.2):
    output_dist = output.data.view(-1).div(temperature).exp()
    top_i = torch.multinomial(output_dist, 1)[0]
    return chr(top_i)

def char_tensor(string):
    string = unidecode(string)
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        try:
            tensor[c] = ord(string[c])
        except:
            print(c)
            raise
    return Variable(tensor)

print(char_tensor('The omega (Ω) symbol\n'))

tensor([ 84, 104, 101,  32, 111, 109, 101, 103,  97,  32,  40,  79,  41,  32,
        115, 121, 109,  98, 111, 108,  10])


In [6]:
# Treina sobre um exemplo (i.e. uma amostragem do texto)

def train(inp, target):
    hidden = rnn.init_hidden()
    rnn.zero_grad()
    loss = 0

    inp = inp.cuda()
    hidden, cell = rnn.init_hidden()
    for c in range(len(inp)):
        output, hidden, cell = rnn(inp[c], hidden, cell)
        loss += loss_metric(output, target[c].unsqueeze(0))

    loss.backward()
    optimizer.step()

    return loss.data.item() / len(inp)


In [7]:
import time

n_epochs = 100000
print_every = 1000
embed_size = 128 # ascii representation
hidden_size = 2048
lr = 0.0001

cuda = torch.device('cuda')

rnn = mLSTM(embed_size, hidden_size, embed_size, embed_size).cuda()

optimizer = torch.optim.Adam(rnn.parameters(), lr=lr)
loss_metric = nn.CrossEntropyLoss()

start = time.time()
all_losses = []
loss_avg = 0

dataset = list(map(char_tensor, reviews.review_comment_message.values))



In [8]:
for epoch in range(16, n_epochs + 1):
    for i, review in enumerate(dataset): 
        if len(review) > 2:
            try:
                loss = train(review[:-1].cuda(), review[1:].cuda())       
            except:
                print(review, len(review))
                raise
                
            loss_avg += loss

            if i % print_every == 0:
                print('[(%d %d%%) %.4f]' % (epoch, epoch / n_epochs * 100, loss))
                print(generate_chunk(predict_len = 50), '\n')

[(1 0%) 4.8739]
O produto'u\#="`Aq
.tIKJ~dZ2zbSi!ezXl/D uN=~% 

[(1 0%) 0.8174]
O produto e compre a entrega a restava e prazo compra de en 

[(1 0%) 0.6210]
O produto de otimo que nao finha e do produto entrega a pro 

[(1 0%) 1.0825]
O produto e para de nem ainda de controle e nao falta de co 

[(1 0%) 0.5688]
O produto chegou antes do prazo embalagem nao compra e entr 

[(1 0%) 1.2697]
O produto nao esperava do confortando e tudo perfeito e por 

[(1 0%) 1.1923]
O produto nao foi entregue antes do prazo e previsto do pro 

[(1 0%) 1.5311]
  produto de entrega foi entregue antes do prazo previsto.

[(1 0%) 0.6744]
O produto e chegou antes do prazo. O produto chegou no praz 

[(1 0%) 1.0473]
O produto nao e funciona nao chegou no prazo. Produto chego 

[(1 0%) 1.5381]
O produto veio com o produto e e entrega dentro do prazo, r 

[(1 0%) 1.0127]
O produto de produto veio dentro do prazo e este produto e  

[(1 0%) 0.5189]
O produto chegou antes do prazo estimado. Comprei 

[(3 0%) 0.6856]
O produto chegou antes do prazo e em perfeitas condicoes. R 

[(3 0%) 0.7795]
O produto nao chegou e ate agora nao recebi o produto. A lo 

[(3 0%) 0.6624]
O produto e bom, pois esta com defeito por enquanto aos cor 

[(3 0%) 0.8876]
O produto e otimo, mas o correio esta com defeito. Sera que 

[(3 0%) 0.4729]
O produto e como eu esperava. A compra foi entregue no praz 

[(3 0%) 1.5413]
O produto foi entregue no prazo e veio conforme prometido,  

[(3 0%) 0.9936]
O produto chegou antes do prazo. Parabens! Super recomendo. 

[(3 0%) 0.3995]
O produto nao chegou no tempo estipuldado e com a data da d 

[(3 0%) 0.1227]
O produto era pra chegar e nao funciona o produto atende a  

[(3 0%) 0.7485]
O produto nao foi entregue a cor errada e a resistencia do  

[(3 0%) 0.8628]
O produto chegou antes do prazo estipulado, porem nao foi e 

[(3 0%) 0.8239]
O produto e bom, porem a embalagem de pessima qualidade na  

[(3 0%) 1.1998]
O produto chegou antes do prazo de entrega, prod

[(6 0%) 0.2023]
O produto e exatamente como esperava, mas com muita transpo 

[(6 0%) 0.2555]
O produto chegou no prazo e em perfeitas condicoes e chegou 

[(6 0%) 0.4219]
O produto chegou faltando a carga do correio, mas este prod 

[(6 0%) 0.8969]
O produto e muito bom. Recomendo a todos. Chegou bem antes  

[(6 0%) 1.1679]
O produto e muito bom, confortavel e com o consumidor
Nao  

[(6 0%) 1.3448]
O produto e bom e entrega super rapida. Recomendo.
Entrega 

[(6 0%) 0.6423]
O produto chegou antes do prazo previsto para entrega do pr 

[(6 0%) 0.7919]
O produto nao chegou em desejario, a embalagem do produto a 

[(6 0%) 1.0946]
O produto chegou com defeito e tamanho da bateria que eu co 

[(6 0%) 0.7685]
O produto chegou antes do prazo e em perfeitas condicoes. R 

[(6 0%) 0.4606]
O produto veio com defeito e nao e o que e mais foi entregu 

[(6 0%) 0.7944]
O produto e bom porem nao consigo realizar a entrega de 500 

[(6 0%) 0.5420]
O produto nao e original e a entrega foi dentro do

[(8 0%) 1.0083]
O produto chegou no prazo e em perfeito estado. So nao enti 

[(8 0%) 1.0786]
O produto foi otimo, entrega rapida. Amei o produto e a ent 

[(8 0%) 0.6605]
O produto chegou antes do prazo, muito bom e recebi o produ 

[(8 0%) 1.6188]
O produto so entregue antes do prazo, produto de qualidade  

[(8 0%) 1.6858]
O produto e bem achei de entrega. Nao recebi o produto em c 

[(8 0%) 0.5120]
O produto veio com a telente e sem a contato com a entrega, 

[(8 0%) 0.1168]
O produto chegou antes do prazo e o produto chegou bem ante 

[(8 0%) 0.7961]
O produto e bom de boa qualidade, e o que eu comprei outro  

[(8 0%) 0.9827]
O produto e entregue no prazo estipulado tudo ok! O produto 

[(8 0%) 0.9751]
O produto e bom de ver constatus diferente do que eu a prim 

[(8 0%) 1.2560]
O produto chegou antes do prazo. Recomendo. Obrigada pela e 

[(8 0%) 0.4670]
O produto veio com uma reclamada em adoreico. Mas nao consi 

[(8 0%) 1.3678]
O produto e exelente e recomendo para minha filh

[(11 0%) 0.4199]
O produto chegou nao estava toula de 3 dias uma de de preco 

[(11 0%) 0.4727]
O produto antes do prazo e entregue no prazo e nao e a loja 

[(11 0%) 1.3892]
O produto nao e chgos e meu recebi a produto e o produto de 

[(11 0%) 1.2067]
O produto e muito recebi o produto dite entregar a meundo e 

[(11 0%) 1.7793]
O produto chegou antes do prazo previsto e a da mas a lanni 

[(11 0%) 0.7328]
O produto veio embalado, mas nao ser uma cosa na montado na 

[(11 0%) 1.2614]
O produto esta da faltando de com a compra de de volcoou no 

[(11 0%) 1.7983]
O produto chegou de antes do prazo, comprei compra, mas a e 

[(11 0%) 1.2339]
O produto ante dias minha mesma da entrega comprar no prazo 

[(11 0%) 0.6516]
O produto ante do prazo porem o produto e entrega antes do  

[(11 0%) 1.4764]
O produto foi entregue antes do prazo e exrelente de qualid 

[(11 0%) 0.7329]
O produto foi entregue no prazo da lannister e entrega e no 

[(11 0%) 0.7530]
O produto veio devio demais do praz

[(13 0%) 0.9630]
O produto foi entregue no produto asperava de contrei os pr 

[(13 0%) 1.4695]
O produto ch entrega na pois sentregar a recomendo e espara 

[(13 0%) 1.6665]
O produto nao foi derico porece de tate entregar com a de n 

[(13 0%) 1.1516]
O produto veio entregouotere que a expela ze mas com tou  u 

[(13 0%) 2.2394]
O produto foi entregue na entrega de so prazo e d mas o pre 

[(13 0%) 2.0937]
O produto na a corta produto neio de entregue estoque com o 

[(13 0%) 0.7955]
O produto chegou antes do prazo, sem da compra feito da Sum 

[(13 0%) 0.1452]
O produto aoda foi ente de enterega. Recomendo.. Entrega o  

[(13 0%) 1.1161]
O produto chegou cera de compra fora e o prazo e esta como  

[(13 0%) 1.8297]
O produto chegou antes do produto s aimos que muito que de  

[(13 0%) 1.5897]
O produto ve aeuerfeito conforme o produto fo  entrega foi  

[(13 0%) 1.4994]
O produto aode compra nao comentes do prazo e embalagem a e 

[(13 0%) 1.2376]
O produto foi entrega rapida. Produ

[(16 0%) 0.1491]
O produto chegou muito boa ter lannister napida e da pra es 

[(16 0%) 0.4862]
O produto e exave no produto ceum Vou reto produto e e otim 

[(16 0%) 0.3693]
O produto chegou antes do prazo, e o que eu sola nas entreg 

[(16 0%) 0.4992]
O produto chegou comprar posse seria recomente a loja, com  

[(16 0%) 1.6795]
O produto chegentro do produto enterar o veio antes do prod 

[(16 0%) 1.4035]
O produto chegou e produto acum empresserci a cabone e no q 

[(16 0%) 2.1469]
O produto chegou com destivel para que nao entrega fora tra 

[(16 0%) 1.0048]
O produto chegou nao to disato parado do prazo. E nao receb 

[(16 0%) 1.3221]
O produto chegou em ante o pedido do prazo. Recebi o produt 

[(16 0%) 1.8214]
O produto de emois so ue enviado, muito do prazo. Por um po 

[(16 0%) 1.5285]
O produto vo que lindo para que fazar mas a entrega foi pro 

[(16 0%) 0.9256]
O produto chegou. entregue muito lente pecto do prazao, ent 

[(16 0%) 1.5727]
O produto cho recebi o produto nao 

KeyboardInterrupt: 

In [10]:
torch.save(rnn.state_dict(), "lstm.pth")